### Step 1. Open the data file and have a look at the general information. 

In [1]:
import pandas as pd
credit_scoring = pd.read_csv('/datasets/credit_scoring_eng.csv')
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
credit_scoring.shape

(21525, 12)

- There are 21525 rows and 12 columns in our dataframe. Two columns `days_employed` and `total_income` have missing values (19351 rows). We have categorical(e.g. `education`), quantitative(e.g. `total_income`) and Boolean-logical (e.g. `debt`) variables. 

In [3]:
# Stats for string data
import numpy as np
credit_scoring.describe(include=np.object)

,education,family_status,gender,income_type,purpose
count,21525,21525,21525,21525,21525
unique,15,5,3,8,38
top,secondary education,married,F,employee,wedding ceremony
freq,13750,12380,14236,11119,797


- According to the statistics above, majority of the customers have secondary education as their highest level of education, are married and female individuals. Customers borrow money mainly for wedding purposes at first glance, but still it is weird that they are only 797 individuals with such purpose and they are at the top, we will dig into that in our further analysis. 

In [4]:
# Top 2 rows of the data
credit_scoring.head(2)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase


In [5]:
# Last 2 rows of the data
credit_scoring.tail(2)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car
21524,2,-1984.507589,40,secondary education,1,married,0,F,employee,0,13127.587,to buy a car


- To this point,we can observe suspicious values in `days_employed` column. The rows have negative values which are illogical to the number of days employed.

### Step 2. Data preprocessing

### Processing missing values

In [6]:
# missing values of dataset
credit_scoring.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
# to see the weight of missing values to total data
missing = credit_scoring.isnull().sum()
total = credit_scoring.notnull().sum()
weight  = ((missing / total)*100).round(2)
print(weight)

children             0.00
days_employed       11.23
dob_years            0.00
education            0.00
education_id         0.00
family_status        0.00
family_status_id     0.00
gender               0.00
income_type          0.00
debt                 0.00
total_income        11.23
purpose              0.00
dtype: float64


In [8]:
# to have an overview of missing value
credit_scoring[credit_scoring['days_employed'].isnull()].head(2)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education


- `days_employed` missing values are missing from `total_income` variable as well and they constitte 11% of total data. Besides, `total_income` has relation with `income_type` column. Thus we can fill the missing values of `total_income` with the average `total_income` according to their `income type`. This approach gives us more accurate data. Round() method is used for quantitative variable to 2 digits.

#### `days_employed` Variable. Number of days employed

In [9]:
# stats of the days employed
credit_scoring['days_employed'].describe().round(2)

count     19351.00
mean      63046.50
std      140827.31
min      -18388.95
25%       -2747.42
50%       -1203.37
75%        -291.10
max      401755.40
Name: days_employed, dtype: float64

- First, we are going to make all the values absolute as we have seen above, some values of `days_employed` column were negative.

In [10]:
# Negative values to absolute
credit_scoring['days_employed'] = abs(credit_scoring['days_employed'])
print(credit_scoring['days_employed'].describe().round(2))
days_employed_median = credit_scoring['days_employed'].median()
days_employed_max = credit_scoring['days_employed'].max()
print('Employment in years: {:.0f}'.format(days_employed_max / 365))

count     19351.00
mean      66914.73
std      139030.88
min          24.14
25%         927.01
50%        2194.22
75%        5537.88
max      401755.40
Name: days_employed, dtype: float64
Employment in years: 1101


 - 1,100 years of employment? There is clearly techinal mistake maybe the definition of the column is not 'days' but other measurement, as well as outliers which push the mean(669,14) up. We will fill it with the median (2,194) since there are significant outliers. 

In [11]:
# Dividing giant numbers by 24, converting to days
to_days =  credit_scoring.loc[(credit_scoring.days_employed > 22000),'days_employed'] / 24
credit_scoring.loc[(credit_scoring.days_employed > 22000),'days_employed'] = to_days
credit_scoring['days_employed'].describe().round(2)

count    19351.00
mean      4641.64
std       5355.96
min         24.14
25%        927.01
50%       2194.22
75%       5537.88
max      18388.95
Name: days_employed, dtype: float64

<font color="orange">  - By taking into account our oldest customer (75 years old) we assume that the maximum years of employment could be 60 years = 22,000 days roughly. So if `days_employed` is bigger than this value, it will be divided by 24, converting them to days. Now the max = 18,388 days(50 years) which is still doubtful but possible. </font>

In [12]:
# Filling missing values with income_type group days_employed average
credit_scoring["days_employed"] = credit_scoring.groupby('income_type')['days_employed'].transform(lambda grp: grp.fillna(np.mean(grp)))
credit_scoring.info()
credit_scoring['days_employed'].isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


0

<font color="orange">  - Missing values from `days_employed` variable were filled with the average days employed by `income_type` variable that gives us more accurate data.</font>

- No missing values in `days_employed` column now. Thats a good news.

#### `total_income` variable. Monthly income 

In [13]:
# total_income stats
missing_total_income = credit_scoring['total_income'].isnull().sum()
print('Missing values:',missing_total_income)
total_income_mean = credit_scoring['total_income'].mean()
total_income_median = credit_scoring['total_income'].median()
print('Total income average: {:.2f}'.format(total_income_mean))
print('Total income median: {:.2f}'.format(total_income_median))


Missing values: 2174
Total income average: 26787.57
Total income median: 23202.87


In [14]:
# stats total_income
credit_scoring['total_income'].describe().round(2)

count     19351.00
mean      26787.57
std       16475.45
min        3306.76
25%       16488.50
50%       23202.87
75%       32549.61
max      362496.64
Name: total_income, dtype: float64

 - There is no any negative value which is good news. The mean is slightly higher than the median which could mean that there are outliers if not significant that pushes mean to the top.

In [15]:
# Filling missing values with income_type group average income
credit_scoring["total_income"] = credit_scoring.groupby('income_type')['total_income'].transform(lambda grp: grp.fillna(np.mean(grp)))
(credit_scoring['total_income'].isnull().sum())
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [16]:
# Output after replacing
credit_scoring['total_income'].describe().round(2)

count     21525.00
mean      26783.35
std       15665.11
min        3306.76
25%       17247.71
50%       24309.01
75%       32386.79
max      362496.64
Name: total_income, dtype: float64

- As we mentioned above, the missing values in `total_income` data will be filled with the total_income mean of each income_type category .transform method. We believe this method of replacing gives us more accuracy. There is no missing valuein our dataset anymore. Great!

<font color='orange'>- Regarding the 0- values. As I understand it is about `children`, `dob_years`, since for `total_income` and `days_employed` stats show ther minimum values.</font>

### Data type replacement

#### `children` variable. Number of children

In [17]:
# to check the number of unique values
credit_scoring['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [18]:
# Negative values to absolute
credit_scoring['children'] = abs(credit_scoring['children'])
credit_scoring['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [19]:
# Replacing outliers
credit_scoring.loc[(credit_scoring.children == 20),'children']= 2
credit_scoring['children'].value_counts()


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

- The reason of replacing the outliers = 20 with 2 is that it is clearly typo error which most probably is 2 rather than    20. .loc method was used to find exact value and replace it. 

In [20]:
# stats children
credit_scoring['children'].describe().round(2)

count    21525.00
mean         0.48
std          0.76
min          0.00
25%          0.00
50%          0.00
75%          1.00
max          5.00
Name: children, dtype: float64

- The table above shows that almost every second person in our dataset has a child. Mean = 0.48

#### `dob_years` variable. Age of the customer

In [21]:
# renaming the column for clarity
credit_scoring.rename(columns={'dob_years':'customer_age'},inplace=True)
credit_scoring['customer_age'].describe().round(2)

count    21525.00
mean        43.29
std         12.57
min          0.00
25%         33.00
50%         42.00
75%         53.00
max         75.00
Name: customer_age, dtype: float64

 - Column's name was renamed using .rename method since it was not clear what exactly `dob_years` mean. The  `customer_age` average  is 43 years. The oldest customer is 75 years old. If we look at min:0. That is out of logic, age can not be 0. Need to analyse and come up with optimal decision. 

In [22]:
# Unique values
credit_scoring['customer_age'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: customer_age, dtype: int64

In [23]:
# Removing 0 from customer_age
to_drop = credit_scoring[credit_scoring['customer_age'] == 0 ]
credit_scoring.drop(to_drop.index, inplace=True)
print(credit_scoring['customer_age'].describe().round(2))
print('The youngest customer is 19 years old.')

count    21424.00
mean        43.50
std         12.25
min         19.00
25%         33.00
50%         43.00
75%         53.00
max         75.00
Name: customer_age, dtype: float64
The youngest customer is 19 years old.


In [24]:
# float to int type to make it more presentable
credit_scoring['days_employed'] = credit_scoring['days_employed'].astype(int)
credit_scoring['total_income'] = credit_scoring['total_income'].astype(int)

### Conclusion
- Negative values were converted to absolute, illogical values were removed since they were of small amount and didnt really affect the overall analysis.
- float to int type to make it easy for further analysis where some calculations might be needed using .astype() method.

### Processing duplicates 

#### - `education` variable.The highest level of education of customers

In [25]:
# check for the unique values
credit_scoring['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

- We can see some values are repeated with UPPERCASE and lowercase letters. We could get rid of them by converting them to lowercase letters

In [26]:
# making all string values lowercase to identify duplicates
credit_scoring['education'] = credit_scoring['education'].str.lower()

In [27]:
# double check
credit_scoring['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

#### `purpose` variable. The reason of taking a loan

In [28]:
# to make sure they are all lowercase
credit_scoring['purpose'] = credit_scoring['purpose'].str.lower()

In [29]:
# Unique purposes to see if we can categorize them 
credit_scoring['purpose'].value_counts()

wedding ceremony                            792
having a wedding                            773
to have a wedding                           769
real estate transactions                    673
buy commercial real estate                  661
buying property for renting out             651
transactions with commercial real estate    649
housing transactions                        647
purchase of the house                       641
housing                                     641
purchase of the house for my family         640
construction of own property                633
transactions with my real estate            630
property                                    630
building a real estate                      623
buy real estate                             621
building a property                         620
purchase of my own house                    619
housing renovation                          610
buy residential real estate                 604
buying my own car                       

In [30]:
# Merging same purpose within the same category 
def purpose_clean(purpose):
    if 'wed' in purpose:
        return 'wedding'
    if 'edu' in purpose:
        return 'education'
    if 'car' in purpose:
        return 'car'
    if 'hous' in purpose:
        return 'real estate'
    if 'estate' in purpose:
        return 'real estate'
    if 'univer' in purpose:
        return 'education'
    if 'property' in purpose:
        return 'real estate'
    return 'other'

credit_scoring['purpose'] = credit_scoring['purpose'].apply(purpose_clean)
credit_scoring['purpose'].unique()

array(['real estate', 'car', 'education', 'wedding'], dtype=object)

In [31]:
credit_scoring['purpose'].value_counts()

real estate    10793
car             4293
education       4004
wedding         2334
Name: purpose, dtype: int64

- Now we have accurate and categorized purposes for taking a  loan.

In [32]:
credit_scoring.duplicated().sum()

404

In [33]:
# to check which values are duplicated. to have a general idea
credit_scoring.loc[credit_scoring.duplicated(), :]

,children,days_employed,customer_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
360,0,2111,27,secondary education,1,married,0,M,business,0,32386,real estate
829,0,2111,57,secondary education,1,civil partnership,1,F,business,0,32386,education
1010,0,15208,66,secondary education,1,married,0,F,retiree,0,21940,real estate
1072,0,2326,44,secondary education,1,civil partnership,1,F,employee,0,25820,real estate
1247,0,2326,54,secondary education,1,civil partnership,1,M,employee,0,25820,wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21364,0,2326,50,secondary education,1,married,0,M,employee,0,25820,real estate
21391,0,2111,52,secondary education,1,married,0,F,business,0,32386,real estate
21414,0,15208,65,secondary education,1,married,0,F,retiree,0,21940,real estate
21415,0,15208,54,secondary education,1,married,0,F,retiree,0,21940,real estate


- We can observe 404 duplicates, which are probably the ones that were NaN values and were replaced. Its better to drop them since the number is not high and for the sake of accuracy.

In [34]:
credit_scoring.drop_duplicates(keep='first', inplace=True)

In [35]:
print('Number of duplicates:' ,credit_scoring.duplicated().sum())
credit_scoring.shape

Number of duplicates: 0


(21020, 12)

### Conclusion

- All the duplicates were dropped. `purpose`'s values were categorized within the same purpose category. `education` .lowercase method was used to avoid duplicated level of education.

### Categorizing Data

- When it comes to categorization and classification, we have to think of how analysis will look like. Categorization & classification is mainly dependent on our furhter analysis.

In [36]:
# Take a look after preprocessing is done
credit_scoring.head(2)

,children,days_employed,customer_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,real estate
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car


In [37]:
# Age_group classification
def age_group(customer_age):
    """
    The function returns the age_group according to the customer_agevalues, using the following rules:
    —'youth' for age  15<= age <= 24
    —'adults' for 25 <= age <= 58
    —'seniors' for all other cases
    """
    
    if customer_age <= 24:
        return 'youth'
    if customer_age <= 58:
        return 'adults'
    return 'seniors'
credit_scoring['age_group'] = credit_scoring['customer_age'].apply(age_group)
credit_scoring.head(2)

,children,days_employed,customer_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,real estate,adults
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car,adults


In [38]:
# to get an idea how to categorize into income levels
credit_scoring['total_income'].describe().round(2)

count     21020.00
mean      26813.98
std       15825.38
min        3306.00
25%       17090.75
50%       24180.00
75%       32386.00
max      362496.00
Name: total_income, dtype: float64

In [39]:
# total_income classification
def income_cat(total_income):
    """
    The function returns the income_group according to the total_income values, using the following rules:
    —'low_income' for total_income  <= 6000
    —'lower_middle_income' for 6001 <= income <= 12000
    —'upper_middle_income' 12001 <= income <= 28000
    — 'high_income'  for all other cases
    """
    if total_income <= 6000:
        return 'low_income'
    if total_income <= 12000:
        return 'lower_middle_income'
    if total_income <=28000:
        return 'upper_middle_income'
    return 'high_income'
credit_scoring['income_cat'] = credit_scoring['total_income'].apply(income_cat)
credit_scoring.head(2)

,children,days_employed,customer_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,income_cat
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,real estate,adults,high_income
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car,adults,upper_middle_income


- Income levels were classified mainly by look at stats (min, max, mean).

In [40]:
# No need for classification as its done already in the dataset
credit_scoring.groupby('family_status')['family_status_id'].value_counts()

family_status      family_status_id
civil partnership  1                    4103
divorced           3                    1183
married            0                   12028
unmarried          4                    2768
widow / widower    2                     938
Name: family_status_id, dtype: int64

In [41]:
# No need for classification as its done already in the dataset
credit_scoring.groupby('education')['education_id'].value_counts()

education            education_id
bachelor's degree    0                5177
graduate degree      4                   6
primary education    3                 282
secondary education  1               14813
some college         2                 742
Name: education_id, dtype: int64

In [42]:
credit_scoring['debt'].value_counts()

0    19289
1     1731
Name: debt, dtype: int64

- Majority value is 0 --> can be interpreted as not defaulted on the loan. Because its kind of illogical to say 19289 of customers has defaulted on the loan.

In [43]:
def debt_default (debt):
    if debt == 0:
        return 'not defaulted'
    if debt == 1:
        return 'defaulted'
credit_scoring['debt_default'] = credit_scoring['debt'].apply(debt_default)
credit_scoring.head(2)

,children,days_employed,customer_age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,income_cat,debt_default
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,real estate,adults,high_income,not defaulted
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car,adults,upper_middle_income,not defaulted


-  `debt` variable is Boolean variable with only two values (0,1)

In [44]:
# Tables for answering the questions
credit_scoring.groupby('debt_default')['children'].value_counts()

debt_default   children
defaulted      0            1056
               1             442
               2             202
               3              27
               4               4
not defaulted  0           12705
               1            4340
               2            1899
               3             300
               4              36
               5               9
Name: children, dtype: int64

In [45]:
credit_scoring['children'].value_counts()

0    13761
1     4782
2     2101
3      327
4       40
5        9
Name: children, dtype: int64

In [46]:
credit_scoring.groupby('debt_default')['family_status'].describe()

,count,unique,top,freq
debt_default,,,,
defaulted,1731,5,married,925
not defaulted,19289,5,married,11103


- There are 1731 customers who have defaulted or couldnt pay the loan. Thats the only 8 % of total customer which good. Lower default rate lower provisions on loans from the bank's perspective

In [47]:
# Stats for string data after clearance and additional columns
import numpy as np
credit_scoring.describe(include=np.object)

,education,family_status,gender,income_type,purpose,age_group,income_cat,debt_default
count,21020,21020,21020,21020,21020,21020,21020,21020
unique,5,5,3,8,4,3,4,2
top,secondary education,married,F,employee,real estate,adults,upper_middle_income,not defaulted
freq,14813,12028,13853,10845,10532,17274,11966,19289


### Conclusion

- When the data is clean and categorized we can proceed to our analysis and questions that need to be answered.

### Step 3. Answer these questions

##### Is there a relation between having kids and repaying a loan on time?

In [48]:
#credit_scoring['debt_children_ratio'] 
print(credit_scoring.pivot_table(index='children', values='debt', aggfunc= ['count', 'sum', 'mean']).round(4))
print()
print('Customers who have 2 and 4 children have highest default rate:10%.')

          count   sum    mean
           debt  debt    debt
children                     
0         13761  1056  0.0767
1          4782   442  0.0924
2          2101   202  0.0961
3           327    27  0.0826
4            40     4  0.1000
5             9     0  0.0000

Customers who have 2 and 4 children have highest default rate:10%.


### Conclusion

- Customers  4 children tend to default on the loan much often than the other group of our dataset, and the default rate = 10.00%. Since we could find out groups with highest default rate, we can answer to the question that: there is a relation between having kids and repaying a loan on time.
- But there is an exception with 5 children in the family with 0.00% default rate. It can be explained by the tiny number of observations in the dataset which is only 9 out of 21020 but still we can include them into more reliable groups {0,3,5} children.
- The difference between default rates of sample group with no child (7.67%) and  default rates of sample groups with 1 child, 2, 3 kids  9.24%, 9.61%, 8.26% respectively are not significant which means the variable `children` does not have a huge impact on paying the loan on time but still there is a difference.

##### Is there a relation between marital status and repaying a loan on time?

In [49]:
credit_scoring.pivot_table(index='family_status', values='debt', aggfunc= ['count', 'sum','mean',]).round(4)

,count,sum,mean
,debt,debt,debt
family_status,,,
civil partnership,4103,386,0.0941
divorced,1183,85,0.0719
married,12028,925,0.0769
unmarried,2768,273,0.0986
widow / widower,938,62,0.0661


### Conclusion

- `unmarried` group of customers have the highest default rate (9.86%). It can be explained that unmarried people who are not in serious relationship have tendency not to pay their loan on time.
- It's interesting to see the default rates of `divorced` and `widow / widower` groups have the lowest default rate:7.19% and 7.69% respectively. They have the least number of people who has ever defaulted.
`Civil partnership` and `married` couples have 9.41% and 7.69% default rates respectively. In general the deviation of default rates between defaulted group of people is not that big. But still marital status plays a role when repaying a loan on time.

##### Is there a relation between income level and repaying a loan on time?

In [50]:
credit_scoring.pivot_table(index='income_cat', values='debt', aggfunc= ['count', 'sum','mean']).round(4)

,count,sum,mean
,debt,debt,debt
income_cat,,,
high_income,7198,536,0.0745
low_income,89,5,0.0562
lower_middle_income,1767,131,0.0741
upper_middle_income,11966,1059,0.0885


### Conclusion

- `upper_middle_income` class are the main demand for the bank loan, followed by `high-income` customers. Accordingly, the default rate of upper_middle_income customers is the highest (8.85%) with `high-income` and `lower-middle-income` groups are in second and third positions with the default rates of 7.45% and 7.41% respectively.
- `low-income` customers ironically have the lowest default rate(5.62%) even if they constitute the smallest portion of the defaulters.
- It is worth to mention that classification of income groups plays an important role here. 

##### How do different loan purposes affect on-time repayment of the loan?

In [51]:
credit_scoring.pivot_table(index='purpose', values='debt', aggfunc= ['count', 'sum','mean']).round(4)

,count,sum,mean
,debt,debt,debt
purpose,,,
car,4250,399,0.0939
education,3946,370,0.0938
real estate,10532,778,0.0739
wedding,2292,184,0.0803


### Conclusion

- Majority of customers took a loan (mortgage) for `real estate` expenses but they have the least default rate (7.39%). `purchasing a car` and `education` purposes share the almost same default rates 9.39% 9.38% respectively,  which are the highest among other purposes. People whose purpose was somehow related to `real estate` have the lowest default rate (7.39%).
- The higher the default rate, the higher the risk for the investment from bank's perspective. There is a logic behin the high default rates, `car` & `education` are the risky investments comparint to real estate which is usually a long-term debt and to get this debt you need collateral and high income which makes it harder to default.

In [52]:
# concluding remark
import numpy as np
credit_scoring.describe(include=np.object)

,education,family_status,gender,income_type,purpose,age_group,income_cat,debt_default
count,21020,21020,21020,21020,21020,21020,21020,21020
unique,5,5,3,8,4,3,4,2
top,secondary education,married,F,employee,real estate,adults,upper_middle_income,not defaulted
freq,14813,12028,13853,10845,10532,17274,11966,19289


### Step 4. General conclusion

<div class="alert alert-warning">The ultimate goal of our analysis was to find out <b>if a customer’s marital status and number of children has an impact on whether they will default on a loan.</b> We had some raw data on customers’ credit worthiness.

<b>Description of the data:</b>

`children`: the number of children in the family <div>
`days_employed`: how long the customer has been working<div>
`customer_age`: the customer’s age<div>
`education`: the customer’s education level<div>
`education_id`: identifier for the customer’s education<div>
`family_status`: the customer’s marital status<div>
`family_status_id`: identifier for the customer’s marital status<div>
`gender`: the customer’s gender<div>
`income_type`: the customer’s income type<div>
`debt`: whether the customer has ever defaulted on a loan<div>
`total_income`: monthly income<div>
`purpose`: reason for taking out a loan<div>
`income_cat`: income level category<div>

Before jumping right into analysis, we had to do data preprocessing, getting the dataset cleaned and accurate. The dataset has all three types of variables, quantitative, categorical and logical.
The data cleaning was started by filling gaps by appropriate columns' average(e.g. average total income by income types was calculated to replace missing values in `total_income` variable. missing values (NaN values in `days_employed` were filled by the average days employed of `income_type`), illogical values(e.g. 20 children per customer) is typo mistake were replaced by value 2, negative values (e.g.negative number of children) were converted to absolute values.
The presence of missing values lead to duplicates most of the time, they were detected and removed. 
Now talking about the data itself, majority of the customers have:
    - secondary education -- 70.4 %
    - married -- 57.2% 
    - employees -- 51.6% 
    - female individuals -- 65.9% 
Customers borrow money mainly for real-estate purposes (50.1%) and most of them are adults, aged between 25 - 58, (81.2%). It is worth to point out that only 8.23% of clients have defaulted on their loan. 
Having a general idea of our clients, we needed to find answer for our main questions which are:
 1. <b>*Is there a relation between having kids and repaying a loan on time?*</b>
    - Our study shows that customers with 4 children tend to default on the loan much often than the other group of our dataset, and the default rate is 10.00%. Since we could find out groups with highest default rate, we can answer to the question that: there is a relation between having kids and repaying a loan on time.But there is an exception with 5 children in the family with 0.00% default rate. It can be explained by the tiny number of observations in the dataset which is only 9 out of 21020 but still we can include them into more reliable groups {0,3,5} children. 
    - The difference between default rates of sample group with no child - 7.67% and  default rates of sample groups with 1 child, 2, 3 kids  9.24%, 9.61%, 8.26% respectively. The difference could be vital when we talk about huge database.
   
 2. <b>*Is there a relation between marital status and repaying a loan on time?*</b>
    - `unmarried` group of customers have the highest default rate (9.86%). It can be explained that unmarried people who are not in serious relationship have tendency not to pay their loan on time.
    - It is interesting to see the default rates of `divorced` and `widow / widower` groups have the lowest default rate: 7.19% and 7.69% respectively. They have the least number of people who has ever defaulted.
    - `Civil partnership` and `married` couples have 9.41% and 7.69% default rates respectively. In general the deviation of default rates between defaulted group of people is not that big. But still marital status plays a role when repaying a loan on time.
    
 3. <b>*Is there a relation between income level and repaying a loan on time?*</b>
    - `upper_middle_income` class are the main demand for the bank loan, followed by high-income customers. Accordingly, the default rate of upper_middle_income customers is the highest (8.85%) with high-income and `lower-middle-income groups` are in second and third positions with the default rates of 7.45% and 7.41% respectively.
    - `low-income` customers ironically have the lowest default rate(5.62%) even if they constitute the smallest portion of the defaulters.
It is worth to mention that classification of income groups plays an important role here.
    
 4. <b>*How do different loan purposes affect on-time repayment of the loan?*</b>
    - Majority of customers took a loan (mortgage) for real estate expenses but they have the least default rate (7.39%). purchasing a car and education purposes share the almost same default rates 9.39% 9.38% respectively, which are the highest among other purposes. People whose purpose was somehow related to real estate have the lowest default rate (7.39%).
    - The higher the default rate, the higher the risk for the investment from bank's perspective. There is a logic behin the high default rates, car and education are the risky investments comparint to real estate which is usually a long-term debt and to get this debt you need collateral and high income which makes it harder to default.
    
<div>In a nutshell, by getting answers for these questions, our hypotheses of marital status and having kid affecting the loan repayment on time are accepted. In each question we were able to identify the least/best respondents.
    
<div>I would like to state that, education level also plays a tremendous role and might have a huge impact on loan repayments of the customers. In addition, this variable drives other variables like income level, purpose of taking a loan. By defining the question of how education level could impact on loan repayment time we could be one step closer towards goal.
</div>
